In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as functional
import sklearn.metrics as metrics
from rdkit import Chem
from rdkit.Chem import rdchem as utils

import numpy as np
from numpy import exp
from numpy.random import normal


In [36]:
base_folder = r'spare_data/'

train_r_file = base_folder + 'train_reactants.sdf'
train_ts_file = base_folder + 'train_ts.sdf'
train_p_file = base_folder + 'train_products.sdf'

test_r_file = base_folder + 'test_reactants.sdf'
test_ts_file = base_folder + 'test_ts.sdf'
test_p_file = base_folder + 'test_products.sdf'

train_r = Chem.ForwardSDMolSupplier(train_r_file, removeHs=False, sanitize=False)
train_r = [x for x in train_r]
train_ts = Chem.ForwardSDMolSupplier(train_ts_file, removeHs=False, sanitize=False)
train_ts = [x for x in train_ts]
train_p = Chem.ForwardSDMolSupplier(train_p_file, removeHs=False, sanitize=False)
train_p = [x for x in train_p]

test_r = Chem.ForwardSDMolSupplier(test_r_file, removeHs=False, sanitize=False)
test_r = [x for x in test_r]
test_ts = Chem.ForwardSDMolSupplier(test_ts_file, removeHs=False, sanitize=False)
test_ts = [x for x in test_ts]
test_p = Chem.ForwardSDMolSupplier(test_p_file, removeHs=False, sanitize=False)
test_p = [x for x in test_p]


In [42]:
# restrict number of products created to 30 for testing

len(train_r[0:100])

100

In [71]:
# figuring out padding
max(mol.GetNumAtoms() for mol in train_r) # = 21, same for ts, p
# min(mol.GetNumAtoms() for mol in train_r) # = 4
# need to get more

# train_r_small = train_r[0:100]
# train_ts_small = train_ts[0:100]
# train_p_small = train_p[0:100]

print(torch.__version__)
# do AE
# then get to grips with PTG


21

In [2]:
from ts_vae.data_processors.grambow_processor import ReactionDataset

base_path = r'data/'
train_r_data = ReactionDataset(base_path, geo_file = 'train_r') 

In [14]:
train_r_data.data

Data(edge_attr=[173336, 4], edge_index=[2, 173336], idx=[6739], pos=[89300, 3], x=[89300, 11], z=[89300])

In [7]:
import torch
import torch.nn as nn
from torch.nn import ReLU

class MLPLayer(nn.Module):
    """ Standard MLP layer. """

    def __init__(self, input_dim, output_dim, activation=ReLU, num_hidden=2):
        super().__init__()
        # fc_layers = fully connected layers
        fc_layers = [nn.Linear(input_dim, input_dim) for hidden_idx in range(num_hidden)]
        fc_layers.append(nn.Linear(input_dim, output_dim))
        self.fc_layers = nn.ModuleList(fc_layers)
        self.num_hidden = num_hidden
        self.activation = activation
        
    def forward(self, x):
        y = x
        for hidden_idx in range(self.num_hidden):
            y = self.fc_layers[hidden_idx](y)
            y = self.activation(y)
        y = self.fc_layers[self.num_hidden](y)
        return y

In [ ]:
import torch
import torch.nn.functional as F

from torch.nn import Linear, ReLU
# from torch_geometric.nn import Sequential

class StandardGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, ):
        super(StandardGNN, self).__init__()
        self.l1 = Linear(dataset.num_features, 6)
        self.l2 = Linear(6, 3)

        self.conv1 = GCNConv(dataset.num_features, , cached=True)


In [ ]:
# notes
num_latent_params = 2 * latent_space_dim
network = nn.Sequential(nn.Linear(data_dim, 300), ..., nn.Linear(400, num_latent_params))
encoder(network)